# Logistic Regression

In [39]:
import numpy as np
import matplotlib.pyplot as plt

In [40]:
import math  # This library is to use normal math functions like exp,sin etc.

In [41]:
# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing data

In [42]:
#below where the file is in gdrive, change with your
data_path = "/content/drive/MyDrive/Colab Notebooks/PRNN_A1/Prnn_datasets/"
dataset = np.loadtxt(data_path + 'PCA_MNIST.csv', delimiter=',',skiprows=1)

In [43]:
dataset.shape

(60000, 11)

First column consists of class information

In [44]:
# Normalising dataset needed to prevent exponent going to zero
for i in range(1,dataset.shape[1]):
  dataset[:,i] = (dataset[:,i]-dataset[:,i].min())/(dataset[:,i].max()-dataset[:,i].min())

## Splitting into test and train dataset

In [45]:
count = 40000

In [46]:
X_train = dataset[0:count,1:]

In [47]:
Y_train =dataset[0:count,0]

In [48]:
X_test = dataset[count:,1:]

In [49]:
Y_test = dataset[count:,0]

In [50]:
classes = 10 # Here number of classes is 10 change it according to your dataset

In [51]:
features = X_train.shape[1]

## Creating logistic regression class

In [52]:
def calc_loss(X,W,j):
  X=X.reshape((X.shape[0],1))
  # calculating probabilities of each class
  e=[]                        # exponent
  g = []                      # probabilities (softmax)
  grad = []                   # gradient addition for this image
  for i in range(classes):
    e.append(math.exp(W[i].T@X))
  k = sum(e)
  for i in range(classes):
    g.append(e[i]/k)
  loss = -1*math.log(g[j])     # loss for 1 image
  for i in range(classes):
    if i!=j:
      grad.append(-g[i]*X.reshape(X.shape[0]))
    else:
      grad.append((1-g[i])*X.reshape(X.shape[0]))
  return loss,grad

In [53]:
def train_epoch1(X,Y,W,features,classes):
  final_loss = 0
  final_grad = np.zeros((features+1,classes))
  for i in range(X.shape[0]):
    loss,grad = calc_loss(X[i,:],W,int(Y[i]))
    final_loss =final_loss + loss
    final_grad = final_grad + np.stack(grad,axis = 1)
  return final_loss,final_grad

In [59]:
class Logistic_regression:
  def __init__(self,epochs = 100,alpha = 10,features = 10 ,classes = 10):
    self.alpha = alpha
    self.epochs = epochs
    self.features=features
    self.classes=classes
    W=[]
    for i in range(10):
      W.append(np.ones((11,1))/11)
    self.W = W

  def train(self,X,Y):
    epochs = self.epochs
    alpha = self.alpha
    W = self.W
    tmp = np.ones((X.shape[0],1))  # Creating augmented data
    X=np.column_stack((X,tmp))
    for i in range(epochs):
      loss,grad = train_epoch1(X,Y,W,self.features,self.classes)
      for j in range(classes):
        gradient = grad[:,j].reshape(grad[:,j].shape[0],1)
        W[j]=W[j]+alpha*gradient/X.shape[0]
      if(i%10==0):
        print('loss in',i,'epoch is',loss/X_train.shape[0])
    self.W = W
    
  def test(self,X,Y):
    tmp = np.ones((X.shape[0],1))  # Creating augmented data
    X=np.column_stack((X,tmp))
    conf_matrix = np.zeros((self.classes,self.classes))
    count = 0
    loss = 0
    W=self.W
    for i in range(X.shape[0]):
      X_1 = X[i,:]
      X_1=X_1.reshape((X_1.shape[0],1))
      # calculating probabilities of each class
      e=[]                        # exponent
      g = []                      # probabilities (softmax)
      for j in range(classes):
        e.append(math.exp(W[j].T@X_1))
      k = sum(e)
      for j in range(classes):
        g.append(e[j]/k)
      k = g.index(max(g))
      if k==Y_test[i]:
        count = count+1
      v = int(Y_test[i])
      conf_matrix[v][k] += 1
      loss = loss - 1*math.log(g[v])
    empirical_risk = loss/X_test.shape[0]
    acc = count*100/X.shape[0]
    return acc,conf_matrix,empirical_risk

  def predict(self,X):
    W=self.W
    Y_pred = np.zeros(X.shape[0])
    tmp = np.ones((X.shape[0],1))  # Creating augmented data
    X=np.column_stack((X,tmp))
    for i in range(X.shape[0]):
      X_1 = X[i,:]
      X_1=X_1.reshape((X_1.shape[0],1))
      # calculating probabilities of each class
      e=[]                        # exponent
      g = []                      # probabilities (softmax)
      for j in range(classes):
        e.append(math.exp(W[j].T@X_1))
      k = sum(e)
      for j in range(classes):
        g.append(e[j]/k)
      k = g.index(max(g))
      Y_pred[i]=k
    return Y_pred

## Running for example dataset

In [61]:
l =Logistic_regression()

In [62]:
l.train(X_train,Y_train)

loss in 0 epoch is 2.3025850929947826
loss in 10 epoch is 2.4709195501334
loss in 20 epoch is 1.982849778896829
loss in 30 epoch is 1.3455708679205158
loss in 40 epoch is 1.2589994454644386
loss in 50 epoch is 0.9745931884858541
loss in 60 epoch is 0.897040369083415
loss in 70 epoch is 0.8108161421246585
loss in 80 epoch is 0.7562949492875214
loss in 90 epoch is 0.7174560438906715


In [63]:
acc,conf_matrix,empirical_risk = l.test(X_test,Y_test)

In [64]:
print("accuracy is:",acc)


accuracy is: 83.745


In [65]:
y_pred = l.predict(X_test)